In [1]:
import os
from pathlib import Path
from ultralytics import YOLO
import torch
import yaml
from PIL import Image
import shutil

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/jean/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PyTorch version: 2.9.1+cu128
CUDA available: False


In [2]:
DATASET_WHEELCHAIR_PATH = Path("Wheelchair-1")
PROJECT_DIR = Path("")
DATASET_DIR = PROJECT_DIR / "dataset"

PROJECT_DIR.mkdir(exist_ok=True)
DATASET_DIR.mkdir(exist_ok=True)

print(f"Using local dataset: {DATASET_WHEELCHAIR_PATH.absolute()}")

Using local dataset: /home/jean/dev/ai_project/Stib4All/detection/Wheelchair-1


In [3]:
%pwd

'/home/jean/dev/ai_project/Stib4All/detection'

In [4]:
classes_file = DATASET_WHEELCHAIR_PATH / "classes.txt"
if classes_file.exists():
    with open(classes_file, "r", encoding="utf-8") as f:
        class_names = [line.strip() for line in f.readlines()]
    print(f"✓ Classes found: {class_names}")
    print(f"✓ Number of classes: {len(class_names)}")
else:
    print(f"⚠️ classes.txt not found at {classes_file}")
    class_names = []

image_files = list(DATASET_WHEELCHAIR_PATH.glob("**/*.jpeg")) + list(DATASET_WHEELCHAIR_PATH.glob("**/*.jpg"))
label_files = [f for f in DATASET_WHEELCHAIR_PATH.glob("**/*.txt") if f.name != "classes.txt"]

print(f"Total images found: {len(image_files)}")
print(f"Total label files found: {len(label_files)}")

✓ Classes found: ['wheelchair']
✓ Number of classes: 1
Total images found: 417
Total label files found: 417


In [5]:
# Verify the dataset structure
print("Dataset structure verification:")
print(f"  Train images: {DATASET_WHEELCHAIR_PATH / 'train' / 'images'}")
print(f"  Train labels: {DATASET_WHEELCHAIR_PATH / 'train' / 'labels'}")
print(f"  Valid images: {DATASET_WHEELCHAIR_PATH / 'test' / 'images'}")
print(f"  Valid labels: {DATASET_WHEELCHAIR_PATH / 'test' / 'labels'}")

# Count files
print(DATASET_WHEELCHAIR_PATH / 'train' / 'images')
train_imgs = len(list((DATASET_WHEELCHAIR_PATH / 'train' / 'images').glob('*')))
train_lbls = len(list((DATASET_WHEELCHAIR_PATH / 'train' / 'labels').glob('*')))
val_imgs = len(list((DATASET_WHEELCHAIR_PATH / 'test' / 'images').glob('*')))
val_lbls = len(list((DATASET_WHEELCHAIR_PATH / 'test' / 'labels').glob('*')))

print(f"\n✓ Files in dataset:")
print(f"  Training: {train_imgs} images, {train_lbls} labels")
print(f"  Validation: {val_imgs} images, {val_lbls} labels")

Dataset structure verification:
  Train images: Wheelchair-1/train/images
  Train labels: Wheelchair-1/train/labels
  Valid images: Wheelchair-1/test/images
  Valid labels: Wheelchair-1/test/labels
Wheelchair-1/train/images

✓ Files in dataset:
  Training: 362 images, 362 labels
  Validation: 55 images, 55 labels


In [6]:
# Define your dataset configuration using the local dataset
data_config = {
    'path': str(DATASET_WHEELCHAIR_PATH.absolute()),  # Root directory
    'train': 'train/images',  # Path to training images
    'val': 'test/images',    # Path to validation images
    'nc': len(class_names),   # Number of classes from classes.txt
    'names': class_names      # Class names from classes.txt
}

# Save configuration to YAML file
yaml_path = DATASET_WHEELCHAIR_PATH / 'data.yaml'
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(data_config, f, default_flow_style=False, allow_unicode=True)

print(f"✓ Configuration saved to: {yaml_path}")
print("\nDataset configuration:")
print(yaml.dump(data_config, default_flow_style=False, allow_unicode=True))

✓ Configuration saved to: Wheelchair-1/data.yaml

Dataset configuration:
names:
- wheelchair
nc: 1
path: /home/jean/dev/ai_project/Stib4All/detection/Wheelchair-1
train: train/images
val: test/images



# Load and train model

In [7]:
# Load a pre-trained YOLOv8 model
# Start with yolov8n (nano) for faster training, or use yolov8m/yolov8l for better accuracy
model = YOLO('yolov8s.pt')  # Automatically downloads if not present

print("Model loaded successfully!")
print(f"Model type: {type(model)}")

# Display model information
model.info()  # Shows model architecture details

Model loaded successfully!
Model type: <class 'ultralytics.models.yolo.model.YOLO'>
YOLOv8s summary: 129 layers, 11,166,560 parameters, 0 gradients, 28.8 GFLOPs


(129, 11166560, 0, 28.816844800000002)

In [8]:
# Training configuration
training_args = {
    'data': str(yaml_path),      # Path to data.yaml
    'epochs': 50,                # Number of epochs (adjust based on dataset size)
    'imgsz': 640,                # Image size
    'batch': 16,                 # Batch size (reduce if GPU memory is limited)
    'patience': 10,              # Early stopping patience
    'save': True,                # Save checkpoints
    'device': 0 if torch.cuda.is_available() else 'cpu',  # Use GPU if available
    'project': str(PROJECT_DIR), # Project directory
    'name': 'yolov8_custom',     # Experiment name
    'exist_ok': True,            # Overwrite existing project
    'pretrained': True,          # Use pretrained weights
    'optimizer': 'AdamW',        # Optimizer (SGD, Adam, AdamW)
    'lr0': 0.001,                # Initial learning rate
    'weight_decay': 0.0005,      # Weight decay
    'warmup_epochs': 3,          # Warmup epochs
    'augment': True,             # Use data augmentation
    'verbose': True,             # Verbose output
}

print("Training configuration:")
for key, value in training_args.items():
    print(f"  {key}: {value}")

print("\n⚠️ Note: Training may take several hours depending on dataset size and hardware!")
print("You can monitor progress in real-time through the console output.")

Training configuration:
  data: Wheelchair-1/data.yaml
  epochs: 50
  imgsz: 640
  batch: 16
  patience: 10
  save: True
  device: cpu
  project: .
  name: yolov8_custom
  exist_ok: True
  pretrained: True
  optimizer: AdamW
  lr0: 0.001
  weight_decay: 0.0005
  warmup_epochs: 3
  augment: True
  verbose: True

⚠️ Note: Training may take several hours depending on dataset size and hardware!
You can monitor progress in real-time through the console output.


In [9]:
# Start training with the local dataset
# Uncomment the following lines when you're ready to train
results = model.train(
    data=str(yaml_path),
    epochs=50,
    imgsz=640,
    batch=16,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=str(PROJECT_DIR),
    name='yolov8_custom',
    exist_ok=True
)

print("✓ Training completed!")
print(f"Results saved to: {PROJECT_DIR / 'yolov8_custom'}")

New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.232 🚀 Python-3.12.3 torch-2.9.1+cu128 CPU (AMD Ryzen 5 7600X 6-Core Processor)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Wheelchair-1/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_custom, nbs=64, nms=False

In [10]:
# Load the best trained model
# After training, the best model is saved as 'best.pt'
best_model_path = PROJECT_DIR / 'yolov8_custom' / 'weights' / 'best.pt'

# Check if model exists
if best_model_path.exists():
    model = YOLO(str(best_model_path))
    print(f"Loaded fine-tuned model from: {best_model_path}")
    
    # Validate the model on the validation set
    metrics = model.val(data=str(yaml_path))
    
    # Display metrics
    print("\n" + "="*50)
    print("Validation Metrics")
    print("="*50)
    print(f"mAP50: {metrics.box.map50:.4f}")        # mAP at IoU=0.50
    print(f"mAP50-95: {metrics.box.map:.4f}")       # mAP at IoU=0.50:0.95
    print(f"Precision: {metrics.box.mp:.4f}")       # Mean precision
    print(f"Recall: {metrics.box.mr:.4f}")          # Mean recall
    print("="*50)
else:
    print(f"Model not found at: {best_model_path}")
    print("Please train the model first!")
    print("\nFor demonstration, you can load the pre-trained model:")
    model = YOLO('yolov8n.pt')

Loaded fine-tuned model from: yolov8_custom/weights/best.pt
Ultralytics 8.3.232 🚀 Python-3.12.3 torch-2.9.1+cu128 CPU (AMD Ryzen 5 7600X 6-Core Processor)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2513.2±975.2 MB/s, size: 46.9 KB)
val: Scanning /home/jean/dev/ai_project/Stib4All/detection/Wheelchair-1/test/labels.cache... 55 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 55/55 109.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.2s/it 4.9s2.1ss
                   all         55         61      0.966      0.934      0.966      0.782
Speed: 0.6ms preprocess, 85.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /home/jean/dev/ai_project/runs/detect/val2

Validation Metrics
mAP50: 0.9657
mAP50-95: 0.7823
Precision: 0.9661
Recall: 0.9336
